### Data Deduplication

In [104]:
import recordlinkage
from recordlinkage.datasets import load_febrl1

In [105]:
dfA = load_febrl1()

dfA.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-223-org,NaN,waller,6,tullaroop street,willaroo,st james,4011,wa,19081209,6988048
rec-122-org,lachlan,berry,69,giblin street,killarney,bittern,4814,qld,19990219,7364009
rec-373-org,deakin,sondergeld,48,goldfinch circuit,kooltuo,canterbury,2776,vic,19600210,2635962
rec-10-dup-0,kayla,harrington,NaN,maltby circuit,coaling,coolaroo,3465,nsw,19150612,9004242
rec-227-org,luke,purdon,23,ramsay place,mirani,garbutt,2260,vic,19831024,8099933


##### Indexer 
Creates the entire combinations of the indexes in dfA

In [106]:
indexer = recordlinkage.Index()
indexer.full()
candidate_links = indexer.index(dfA)

In [107]:
type(candidate_links)

pandas.core.indexes.multi.MultiIndex

Below we are printing the number or rows we have and the number of combinations we have

In [108]:
print(len(dfA), len(candidate_links))

1000 499500


Another method for indexing is called blockage. In this case it is automatically grouping together any instances where the given names are identical or more stored attributes of the person (or entity). 

In [109]:
indexer = recordlinkage.Index()
indexer.block(['given_name','date_of_birth'])
candidate_links = indexer.index(dfA)

print (len(candidate_links))

290


In [110]:
candidate_links

MultiIndex([('rec-122-dup-0',   'rec-122-org'),
            ('rec-373-dup-0',   'rec-373-org'),
            (   'rec-10-org',  'rec-10-dup-0'),
            ('rec-294-dup-0',   'rec-294-org'),
            (   'rec-81-org',  'rec-81-dup-0'),
            ('rec-225-dup-0',   'rec-225-org'),
            ('rec-452-dup-0',   'rec-452-org'),
            ( 'rec-67-dup-0',    'rec-67-org'),
            ('rec-286-dup-0',   'rec-286-org'),
            (  'rec-251-org', 'rec-251-dup-0'),
            ...
            ('rec-298-dup-0',   'rec-298-org'),
            ('rec-345-dup-0',   'rec-345-org'),
            ( 'rec-16-dup-0',    'rec-16-org'),
            (  'rec-318-org', 'rec-318-dup-0'),
            ('rec-437-dup-0',   'rec-437-org'),
            (  'rec-282-org', 'rec-282-dup-0'),
            ('rec-299-dup-0',   'rec-299-org'),
            (  'rec-132-org', 'rec-132-dup-0'),
            (    'rec-8-org',   'rec-8-dup-0'),
            (   'rec-83-org',  'rec-83-dup-0')],
           names=['rec_

In [111]:
dfA.loc['rec-122-dup-0']

given_name             lachlan
surname                  berry
street_number               69
address_1        giblin street
address_2            killarney
suburb                 bittern
postcode                  4184
state                      qld
date_of_birth         19990219
soc_sec_id             7364009
Name: rec-122-dup-0, dtype: object

In [112]:
dfA.loc['rec-122-org']

given_name             lachlan
surname                  berry
street_number               69
address_1        giblin street
address_2            killarney
suburb                 bittern
postcode                  4814
state                      qld
date_of_birth         19990219
soc_sec_id             7364009
Name: rec-122-org, dtype: object

In [113]:
indexer = recordlinkage.Index()
indexer.full()
candidate_links = indexer.index(dfA)

To make record pairs now, we will have to 

In [ ]:
# This cell can take some time to compute.
compare_cl = recordlinkage.Compare()

compare_cl.string('given_name', 'given_name', threshold = 0.85, label='given_name')
compare_cl.string('surname', 'surname', method='jarowinkler', threshold=0.85, label='surname')
compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_cl.exact('suburb', 'suburb', label='suburb')
compare_cl.exact('state', 'state', label='state')
compare_cl.string('address_1', 'address_1', threshold=0.85, label='address_1')

features = compare_cl.compute(candidate_links, dfA)

In [ ]:
features.head(10)

In [ ]:
# Sum the comparison results.
features.sum(axis=1).value_counts().sort_index(ascending=False)

In [ ]:
# Classification step
matches = features[features.sum(axis=1) > 1]
print(len(matches))

In [ ]:
matches

### Comparisons Between Data Sources

In [ ]:
from recordlinkage.datasets import load_febrl4
dfA, dfB = load_febrl4()

dfA

In [ ]:
dfB

In [ ]:
indexer = recordlinkage.Index()
indexer.full()
pairs = indexer.index(dfA, dfB)

In [ ]:
indexer = recordlinkage.Index()
indexer.block('given_name')
candidate_links = indexer.index(dfA, dfB)

print (len(candidate_links))

In [ ]:
print (len(dfA), len(dfB), len(pairs))

In [ ]:
# This cell can take some time to compute.
compare_cl = recordlinkage.Compare()

compare_cl.exact('given_name', 'given_name', label='given_name')
compare_cl.string('surname', 'surname', method='jarowinkler', threshold=0.85, label='surname')
compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_cl.exact('soc_sec_id','soc_sec_id',label='soc_sec_id')
compare_cl.exact('suburb', 'suburb', label='suburb')
compare_cl.exact('state', 'state', label='state')
compare_cl.string('address_1', 'address_1', threshold=0.85, label='address_1')

features = compare_cl.compute(candidate_links, dfA, dfB)

In [ ]:
# Sum the comparison results.
features.sum(axis=1).value_counts().sort_index(ascending=False)

In [ ]:
features[features.sum(axis=1) > 1]

In [ ]:
dfB.loc['rec-4652-dup-0']

In [ ]:
dfA.loc['rec-2371-org']

### Another Method
Another option that I discussed with Olena was to use these techniques to find some actual matches and non-matches and then use a sample to run a random forest with those distance metrics and then create a model that way to match up certain ones. 